# 📘 Row-by-Row Explanation


In [ ]:

# 📦 Importing libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt

# 📁 Importing data
Df = pd.read_csv('ICICI Minute Data.csv')

# 🧹 Removing zero-volume rows (likely market not active)
Df = Df.drop(Df[Df['Volume'] == 0].index)

# ⏱️ Converting time string to datetime
Df['Time'] = pd.to_datetime(Df['Time'])

# 📊 Splitting 80% for training
t = .8
split = int(t * len(Df))

# 📈 Creating technical indicators
n = 10
Df['RSI'] = ta.RSI(np.array(Df['Close'].shift(1)), timeperiod=n)
Df['SMA'] = Df['Close'].shift(1).rolling(window=n).mean()
Df['Corr'] = Df['Close'].shift(1).rolling(window=n).corr(Df['SMA'].shift(1))
Df['SAR'] = ta.SAR(np.array(Df['High'].shift(1)), np.array(Df['Low'].shift(1)), 0.2, 0.2)
Df['ADX'] = ta.ADX(np.array(Df['High'].shift(1)), np.array(Df['Low'].shift(1)), np.array(Df['Open']), timeperiod=n)

# 🔁 Creating lagged features for price movement
Df['open'] = Df['Open'].shift(1)
Df['close'] = Df['Close'].shift(1)
Df['high'] = Df['High'].shift(1)
Df['low'] = Df['Low'].shift(1)
Df['OO'] = Df['Open'] - Df['open']
Df['OC'] = Df['Open'] - Df['close']

# 💹 Calculating returns and past returns
Df['Ret'] = (Df['Open'].shift(-1) - Df['Open']) / Df['Open']
for i in range(1, n):
    Df[f'return{i}'] = Df['Ret'].shift(i)

# 🧼 Fixing invalid correlations and removing NaNs
Df.loc[Df['Corr'] < -1, 'Corr'] = -1
Df.loc[Df['Corr'] > 1, 'Corr'] = 1
Df = Df.dropna()

# 🎯 Creating Signal column for classification
Df['Signal'] = 0
Df.loc[Df['Ret'] > Df['Ret'][:split].quantile(0.66), 'Signal'] = 1
Df.loc[Df['Ret'] < Df['Ret'][:split].quantile(0.34), 'Signal'] = -1

# 🕒 Neutral signal at market close time
Df.loc[(Df['Time'].dt.hour == 15) & (Df['Time'].dt.minute == 29), 'Signal'] = 0
Df.loc[(Df['Time'].dt.hour == 15) & (Df['Time'].dt.minute == 29), 'Ret'] = 0

# 🧠 Feature selection (dropping non-predictive columns)
X = Df.drop(['Close', 'Signal', 'Time', 'High', 'Low', 'Volume', 'Ret'], axis=1)
y = Df['Signal']

# ⚙️ ML Pipeline
steps = [('scaler', StandardScaler()), ('svc', SVC())]
pipeline = Pipeline(steps)

# 🔍 Hyperparameter tuning with RandomizedSearchCV
params = {'svc__C': [10, 100, 1000, 10000],
          'svc__gamma': [1e-2, 1e-1, 1e0],
          'svc__kernel': ['rbf']}
rcv = RandomizedSearchCV(pipeline, params, cv=7)
rcv.fit(X.iloc[:split], y.iloc[:split])

# 📌 Best parameters
best_C = rcv.best_params_['svc__C']
best_gamma = rcv.best_params_['svc__gamma']
best_kernel = rcv.best_params_['svc__kernel']

# 🎓 Train final model with best parameters
cls = SVC(C=best_C, kernel=best_kernel, gamma=best_gamma)
ss1 = StandardScaler()
cls.fit(ss1.fit_transform(X.iloc[:split]), y.iloc[:split])

# 🔮 Predict on test data
y_predict = cls.predict(ss1.transform(X.iloc[split:]))

# 🧾 Save predictions
Df['Pred_Signal'] = 0
Df.iloc[:split, Df.columns.get_loc('Pred_Signal')] = cls.predict(ss1.transform(X.iloc[:split]))
Df.iloc[split:, Df.columns.get_loc('Pred_Signal')] = y_predict

# 💰 Strategy return
Df['Ret1'] = Df['Ret'] * Df['Pred_Signal']
excess_strategy_returns = Df['Ret1'][split:]

# 📈 Sharpe ratio calculation (6.25 hours × 60 mins = 375 min trading day)
Sharpe = np.sqrt(252 * 375) * excess_strategy_returns.mean() / excess_strategy_returns.std()
print("Sharpe Ratio:", Sharpe)
